## GED conflict data clean up

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
##set a working directory
user = os.path.expanduser('~')
display(user)

os.makedirs(f'{user}/Desktop/thesis/', exist_ok=True)
path = os.chdir(f'{user}/Desktop/'+'thesis/')

'/Users/NatStrom'

In [3]:
data = pd.read_csv('GEDEvent_v22_1.csv')

/var/folders/3l/pgxm2f9j3bbgbjcj2pyxl__00000gp/T/ipykernel_11394/2663555733.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('GEDEvent_v22_1.csv')


In [4]:
df = data.copy()

In [5]:
print(df.dtypes)
print(df.columns)
print(df.describe(include='all'))

## sanity check to ensure the spread of the data is correct
print('sanity check 1, ensure appropriate years and countries are represented')
list_country=list(list(df['country_id'].drop_duplicates()))
list_year=list(list(df['year'].drop_duplicates()))
print('country list', list_country)
print('year list', list_year)
#inspect for NAs
print('following checks for completeness of all columns')
print(df['country_id'].isnull().values.any())

id                     int64
relid                 object
year                   int64
active_year            int64
code_status           object
type_of_violence       int64
conflict_dset_id       int64
conflict_new_id        int64
conflict_name         object
dyad_dset_id           int64
dyad_new_id            int64
dyad_name             object
side_a_dset_id         int64
side_a_new_id          int64
side_a                object
side_b_dset_id         int64
side_b_new_id          int64
side_b                object
number_of_sources      int64
source_article        object
source_office         object
source_date           object
source_headline       object
source_original       object
where_prec             int64
where_coordinates     object
where_description     object
adm_1                 object
adm_2                 object
latitude             float64
longitude            float64
geom_wkt              object
priogrid_gid           int64
country               object
country_id    

In [6]:
#filter only the wanted columns
df_filtered = df.filter(['id', 'relid', 'year', 'type_of_violence','conflict_new_id','country_id','best', 'high', 'low'])

#df_filtered = 
#add the iso codes


In [7]:
display(df_filtered)

,id,relid,year,type_of_violence,conflict_new_id,country_id,best,high,low
0,244657,IRQ-2017-1-524-322,2017,1,259,700,6,6,6
1,412700,IRQ-2021-1-524-145,2021,1,259,700,183,184,171
2,413023,IRQ-2021-1-524-143,2021,1,259,700,2,3,0
3,412909,IRQ-2021-1-524-144,2021,1,259,700,10,10,9
4,132140,AFG-1989-1-411-2,1989,1,333,700,6,6,6
...,...,...,...,...,...,...,...,...,...
293629,15525,MZM-1989-3-1347-4,1989,3,562,552,9,9,9
293630,15524,MZM-1989-3-1347-16,1989,3,562,552,9,9,9
293631,15245,MZM-1990-3-1347-18,1990,3,562,552,7,7,7
293632,15516,MZM-1990-3-1347-28,1990,3,562,552,1,1,1


In [8]:
from ingester3.scratch import cache_manager

cache_manager()

# This will import the Pandas extensions as well as bootstrap the object layer
from ingester3.extensions import *

In [9]:
#create the c_id from the gw codes
df_filtered = pd.DataFrame.c.from_gwcode(df_filtered, gw_col='country_id', month_col=None)

In [10]:
df_conflict = df_filtered.copy()

In [11]:
df_conflict = df_conflict.filter(['year', 'type_of_violence','c_id', 'best', 'high', 'low'])
df_conflict

,year,type_of_violence,c_id,best,high,low
0,2017,1,133,6,6,6
1,2021,1,133,183,184,171
2,2021,1,133,2,3,0
3,2021,1,133,10,10,9
4,1989,1,133,6,6,6
...,...,...,...,...,...,...
293629,1989,3,158,9,9,9
293630,1989,3,158,9,9,9
293631,1990,3,158,7,7,7
293632,1990,3,158,1,1,1


In [12]:
#replace row values
df_conflict['type_of_violence'] = df_conflict['type_of_violence'].map({1:'sb',
                                            2:'ns',
                                            3:'os'})

In [13]:
#pivot the type of violence
#pivoting of the df to resemble the VIEWS structure

df_pivot = df_conflict.pivot_table(index=['c_id','year'],columns=['type_of_violence'],values=['best'],  aggfunc= 'sum')

df_pivot.assign(**df_pivot.index.to_frame()) #need to do this if the data is multihierarchical

df_pivot.columns = ['_'.join(col) for col in df_pivot.columns]
df_pivot=df_pivot.reset_index()
df_pivot

,c_id,year,best_ns,best_os,best_sb
0,1,2008,NaN,29.0,NaN
1,3,1990,NaN,NaN,39.0
2,4,1989,NaN,58.0,NaN
3,4,1990,NaN,64.0,NaN
4,4,1991,NaN,30.0,NaN
...,...,...,...,...,...
1823,246,2017,533.0,254.0,484.0
1824,246,2018,480.0,194.0,543.0
1825,246,2019,409.0,118.0,115.0
1826,246,2020,656.0,28.0,64.0


In [14]:
df_conflict = df_pivot.copy()
df_conflict = df_conflict.groupby(by=['c_id','year']).sum().reset_index()
df_conflict['best_all'] = df_conflict['best_sb'] + df_conflict['best_ns'] + df_conflict['best_os']
df_conflict

,c_id,year,best_ns,best_os,best_sb,best_all
0,1,2008,0.0,29.0,0.0,29.0
1,3,1990,0.0,0.0,39.0,39.0
2,4,1989,0.0,58.0,0.0,58.0
3,4,1990,0.0,64.0,0.0,64.0
4,4,1991,0.0,30.0,0.0,30.0
...,...,...,...,...,...,...
1823,246,2017,533.0,254.0,484.0,1271.0
1824,246,2018,480.0,194.0,543.0,1217.0
1825,246,2019,409.0,118.0,115.0,642.0
1826,246,2020,656.0,28.0,64.0,748.0


In [15]:
df_conflict.to_csv('df_conflict.csv')